In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
from google.colab import drive, files # google colab specific
import requests
import pandas as pd
import os
import warnings
import sys
import matplotlib.pyplot as plt
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow import keras
from keras import layers
import math
from sklearn.metrics import mean_squared_error
from datetime import datetime
import datetime as dt
from pylab import rcParams
from numpy import array


In [3]:
# Read Data from CSV into Dataframe
df = pd.read_csv('/content/drive/MyDrive/JPM/jphomevalue/data/MonthlyHousePriceDataTampa.csv')
df.head()

,DATE,TPXRSA,"3511 E 29th Ave, Tampa, FL 33605","214 W Emily St, Tampa, FL 33603","2904 W Averill Ave, Tampa, FL 33611","2911 W Estrella St, Tampa, FL 33629","4118 W Olive St, Tampa, FL 33616","8716 N Greenwood Ave, Tampa, FL 33604","2412 E 20th Ave, Tampa, FL 33605","5828 Mariner St, Tampa, FL 33609",...,"2601 W Hillsborough Ave, Tampa, FL 33614","1022 S Frankland Rd, Tampa, FL 33629","105 E 26th Ave, Tampa, FL 33603","5120 W Poe Ave, Tampa, FL 33629","Tampa Bay Blvd, Tampa, FL 33614","4209 W Cayuga St, Tampa, FL 33614","6407 S Clark Ave, Tampa, FL 33616","15215 Amberly Dr APT 1008, Tampa, FL 33647","821 E McEwen Ave, Tampa, FL 33612","8512 N 11th St, Tampa, FL 33604"
0,2002-01-31,120.690219,58977.28716,64689.95754,162770.8757,260751.2188,104115.4292,65655.47929,28362.20153,744175.8921,...,358570.6415,1651645.651,75250.35173,1680611.304,321277.3638,329363.6084,154966.2416,58414.06614,46827.80508,68983.08105
1,2002-02-28,122.877500,60046.13828,65862.33994,165720.7882,265476.8385,106002.3232,66845.35994,28876.21248,757662.6644,...,365069.0522,1681578.586,76614.12119,1711069.186,327099.9047,335332.6972,157774.7099,59472.70995,47676.46996,70233.26816
2,2002-03-31,123.682889,60439.70520,66294.02862,166806.9899,267216.8822,106697.1058,67283.49174,29065.47897,762628.6949,...,367461.8639,1692600.339,77116.28143,1722284.232,329243.8511,337530.6047,158808.8298,59862.51838,47988.96102,70693.60568
3,2002-04-30,124.908879,61038.80550,66951.15911,168460.4414,269865.6329,107754.7262,67950.43014,29353.58655,770188.1475,...,371104.2793,1709378.008,77880.68601,1739356.139,332507.4357,340876.3306,160383.0005,60455.89740,48464.64502,71394.34637
4,2002-05-31,125.576795,61365.19399,67309.16230,169361.2380,271308.6663,108330.9154,68313.77666,29510.54690,774306.5200,...,373088.6589,1718518.444,78297.13189,1748656.875,334285.4292,342699.0745,161240.6052,60779.16894,48723.79659,71776.10831


In [12]:
# ********* PRIOR INITIALIZATIONS **************************
# *************** RUN ONCE *********************************

# Load the model to be used
model = keras.models.load_model('/content/drive/MyDrive/JPM/jphomevalue/trained_models/lstm-model-tampa-24-15.h5')

# Create a skeleton dataframe that will be initialized with Daily Shiller Data
shiller_filePath = "/content/drive/MyDrive/JPM/jphomevalue/data/Shiller12monthpredicted.csv"
shiller_data = pd.read_csv(shiller_filePath, header = 0)

m_df = shiller_data.copy()

# Scaler
scaler = MinMaxScaler(feature_range=(0,1))


In [ ]:
# ********* AUTOMATION BEGINS HERE! ************************

# Each column represents a house and it's historical prices
# Isolate a Date|House pair and store it into another df

for i in range(df.shape[1]-1): 
   # Store Date|House Pair and store to selective_df
   selective_df = df.iloc[:, 2+i]

   # Store the current column name ('current house address)
   current_house = df.columns.values[2+i]

   # Call auto_prediction() function to automatically predict new prices and append to a new dataframe
   prediction_data = auto_prediction(selective_df, current_house) 
   #prediction = predictions.tolist()
   print('Returned from auto pred function')
   df_temp = selective_df.tolist()
   df_temp.extend(prediction_data)

   # Start appending to new main dataframe
   m_df[current_house] = df_temp


# After the new dataframe is completely filled with prediction data transform to CSV  
m_df.to_csv('dataTest.csv')

# Download final CSV file and it is ready to populate to DB
files.download('dataTest.csv')

  


In [19]:
# After the new dataframe is completely filled with prediction data transform to CSV  
m_df.to_csv('completeHousePriceDataTampa.csv')

# Download final CSV file and it is ready to populate to DB
files.download('completeHousePriceDataTampa.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# auto_prediction() function

def auto_prediction(house_data, house_address): 
  
  # Normalize and Reshape the house data passed
  house_data = scaler.fit_transform(np.array(house_data).reshape(-1,1))

  # Use the last 24 entries of the house_data for prediction
  prediction_input = house_data[int(house_data.shape[0])-24:int(selective_df.shape[0]),:].reshape(1,-1)

  # Call the prediction generator function to return the new predictions
  new_predictions = prediction_generator(prediction_input)
  
  return new_predictions



In [16]:
# prediction_generator() function

def prediction_generator(pred_input_data):

    pred_input = pred_input_data
    # Add pred_input to temp_input
    temp_input = []
    temp_input=list(pred_input)
    temp_input=temp_input[0].tolist()

    # Predict timesteps in the future
    list_output=[]
    n_steps=24
    k=0

    while(k<12):
        
        if(len(temp_input)>24):
            print('Entering main condition')
            #print(temp_input)
            pred_input=np.array(temp_input[1:]) # Shift one to the right
            print("{} month input {}".format(k,pred_input))
            pred_input=pred_input.reshape(1,-1)
            pred_input = pred_input.reshape(1, n_steps, 1)
            print('Pred_input shape is:', pred_input.shape)
            fut_pred = model.predict(pred_input, verbose=0)
            print("{} month output {}".format(k,fut_pred))
            temp_input.extend(fut_pred[0].tolist()) # Add that prediction to the temp input list as that will be used in the next prediction 
            temp_input=temp_input[1:] #Shift the temp_input to remove the first element which was already used in a sequence
            #print(temp_input)
            list_output.extend(fut_pred.tolist())
            k=k+1
        else:
            print('Entering else condition')
            pred_input = pred_input.reshape((1, n_steps,1))
            fut_pred = model.predict(pred_input, verbose=0)
            print('The predicted value is:',fut_pred[0])
            temp_input.extend(fut_pred[0].tolist())
            print(len(temp_input))
            list_output.extend(fut_pred.tolist())
            k=k+1
        

    list_output = np.array(scaler.inverse_transform(list_output))
    returned_output = list_output[:,0]
    return returned_output



